# Tutorial 05: weak imposition of Dirichlet BCs by a Lagrange multiplier (inf-sup computation)

In this tutorial we compute the inf-sup constant of the saddle point problem resulting from the discretization of the following Laplace problem
$$\begin{cases}
-\Delta u = f, & \text{in } \Omega,\\
 u   = 0, & \text{on } \Gamma = \partial\Omega,
\end{cases}$$

where $\Omega$ is the unit ball in 2D, and for which the non-homogeneous Dirichlet boundary conditions are imposed by a Lagrange multiplier.

The resulting eigenvalue problem is
$$
\text{find } \eta, u, \lambda \in \mathbb{R} \times V \times M \text{ s.t. }\\
\begin{cases}
\int_\Omega \nabla u \cdot \nabla v + \int_\Gamma \lambda v = 0, & \forall v \in V,\\
\int_\Gamma u \mu = \eta \int_\Gamma \lambda \mu, & \forall \mu \in M
\end{cases}
$$
where
$$
V = H^1(\Omega),\\
M = L^{2}(\Gamma).\\
$$

In [ ]:
import dolfinx.fem
import dolfinx.io
import dolfinx.mesh
import mpi4py
import numpy as np
import petsc4py
import slepc4py
import ufl

In [ ]:
import multiphenicsx.fem

### Mesh

In [ ]:
with dolfinx.io.XDMFFile(mpi4py.MPI.COMM_WORLD, "data/circle.xdmf", "r") as infile:
    mesh = infile.read_mesh()
    mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
    boundaries = infile.read_meshtags(mesh, name="boundaries")
facets_Gamma = boundaries.indices[boundaries.values == 1]

### Eigenvalue problem

In [ ]:
# Define a function space
V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 2))

In [ ]:
# Define restrictions.
dofs_V = np.arange(0, V.dofmap.index_map.size_local + V.dofmap.index_map.num_ghosts)
dofs_V_Gamma = dolfinx.fem.locate_dofs_topological(V, boundaries.dim, facets_Gamma)
restriction_V = multiphenicsx.fem.DofMapRestriction(V.dofmap, dofs_V)
restriction_V_Gamma = multiphenicsx.fem.DofMapRestriction(V.dofmap, dofs_V_Gamma)
restriction = [restriction_V, restriction_V_Gamma]

In [ ]:
# Define trial and test functions
(u, l) = (ufl.TrialFunction(V), ufl.TrialFunction(V))
(v, m) = (ufl.TestFunction(V), ufl.TestFunction(V))

In [ ]:
# Define problem block forms
a = [[ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx, - ufl.inner(l, v) * ufl.ds],
     [- ufl.inner(u, m) * ufl.ds, None]]
b = [[None, None],
     [None, - ufl.inner(l, m) * ufl.ds]]
b[0][0] = dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(0)) * ufl.inner(u, v) * ufl.dx

In [ ]:
# Assemble lhs and rhs matrices
A = multiphenicsx.fem.assemble_matrix_block(a, bcs=[], restriction=(restriction, restriction))
A.assemble()
B = multiphenicsx.fem.assemble_matrix_block(b, bcs=[], restriction=(restriction, restriction))
B.assemble()

In [ ]:
# Solve
eps = slepc4py.SLEPc.EPS().create(mesh.comm)
eps.setOperators(A, B)
eps.setProblemType(slepc4py.SLEPc.EPS.ProblemType.GNHEP)
eps.setDimensions(1, petsc4py.PETSc.DECIDE, petsc4py.PETSc.DECIDE)
eps.setWhichEigenpairs(slepc4py.SLEPc.EPS.Which.TARGET_REAL)
eps.setTarget(1.e-5)
eps.getST().setType(slepc4py.SLEPc.ST.Type.SINVERT)
eps.getST().getKSP().setType("preonly")
eps.getST().getKSP().getPC().setType("lu")
eps.getST().getKSP().getPC().setFactorSolverType("mumps")
eps.solve()
assert eps.getConverged() >= 1

In [ ]:
# Extract leading eigenvalue
eigv = eps.getEigenvalue(0)
r, i = eigv.real, eigv.imag
assert abs(i) < 1.e-10
assert r > 0., "r = " + str(r) + " is not positive"
print("Inf-sup constant: ", np.sqrt(r))
assert np.isclose(np.sqrt(r), 0.125496)